# Лаба 2 - Линейная регрессия

## Задача
В этом наборе данных вам предстоит решить актуальную задачу для поисковиков: **нужно оценить насколько привлекателен веб-ресурс по некоторому набору факторов**.

В наборе данных представлено 8000 записей о различных анонимизированных доменах и соответствующие оценки привлекательности(числовые)

Нужно понять насколько домен привлекателен по остальным факторам.

## Описание столбцов

| столбец                | описание                                            |
|------------------------|-----------------------------------------------------|
| category               | категория к которой относится сайт                  |
| clicks                 | кол-во кликов по домену                             |
| likes                  | кол-во лайков поставленных домену                   |
| buys                   | кол-во покупок совершенных на домене                |
| 4xx_errors             | кол-во ошибок с кодом 4хх за последние 6 мес        |
| 5xx_errors             | кол-во ошибок с кодом 5хх за последние 6 мес        |
| complaints_count       | кол-во жалоб на домен                               |
| average_dwelltime      | среднее время проведенное пользователем на домене ( в минутах) |
| date_of_registration   | дата регистрации домена                             |
| source_attractiveness  | привлекательность домена (таргет)                   |

### Пояснение к задаче
Ваша задача предстоит не только в написании кода обучения модели. Глобально вы должны предоставить ноутбук с полноценным анализом данных, очисткой данных, сделать выводы на основе графического анализа.

За отсутствие надлежащего оформления будут снижаться баллы. В критических случаях - лаба не будет принята.

In [15]:
import pandas as pd
# pd.options.display.max_rows = None

import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn import metrics

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [16]:
train_data_path = 'train.csv'
test_data_path = 'test.csv'

In [17]:
initial_df = pd.read_csv(train_data_path)
initial_df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
initial_df.set_index('id', inplace=True)

Введем пару функций для удобной чистки данных:
- `non_numeric_to_minus_one` - приводит все к числам, к -1, чтобы потом удалять нечисленные значения
- `minus_to_zero_or_other` - приводит минус ('-') к нулю, так как посмотрев на датасет, оказалось, что некоторые значения, которые скорее всего равны нулю, были обозначены черточкой, остальные значения обрабатывает переданной в аргументе функцией
- `non_numeric_to_zero` - применял для всех фич, которые не могли быть меньше нуля или не могли быть не числовыми

In [18]:
def non_numeric_to_minus_one(row):
    try:
        value = float(row)
        return value if value > 0 else -1
    except: return -1

def minus_to_zero_or_other(row, other):
    if row == '-': return 0
    return other(row)
    
def non_numeric_to_zero(row):
    try:
        value = float(row)
        return value if value > 0 else 0
    except: return 0

Натренируем модель на начальном датасете, из новых фич добавим только "возраст" сайта *(потому что по-другому не натренировать модель)*, все `null`ы тупо убираем, потому что лень

In [19]:
default_df = initial_df.copy(deep=True)
print(f'До чистки данных: {default_df.shape}')

default_df.dropna(inplace=True)

default_df['date_of_registration'] = pd.to_datetime(default_df['date_of_registration'])
default_df['age'] = (pd.Timestamp('2024-09-30') - default_df['date_of_registration']) / pd.Timedelta(days=365)
default_df.drop('date_of_registration', axis='columns', inplace=True)

default_df['complaints_count'] = default_df['complaints_count'].apply(non_numeric_to_zero)

print(f'После чистки данных: {default_df.shape}')

default_x_train, default_x_test, default_y_train, default_y_test = train_test_split(
    default_df.drop('source_attractiveness', axis='columns'),
    default_df['source_attractiveness'],
    test_size=0.2,
    shuffle=True,
    stratify=default_df['category']
)

encoder = OneHotEncoder(drop='first', sparse_output=False)
default_x_train = np.hstack([default_x_train.drop(['category'], axis='columns'), encoder.fit_transform(default_x_train['category'].to_frame())])
default_x_test = np.hstack([default_x_test.drop(['category'], axis='columns'), encoder.transform(default_x_test['category'].to_frame())])

default_model = LinearRegression(fit_intercept=True)
default_model.fit(default_x_train, default_y_train)

До чистки данных: (8000, 10)
После чистки данных: (6931, 10)


LinearRegression()

In [20]:
default_test_predicts = default_model.predict(default_x_test)
print(f'TEST:\n\
    MSE = {metrics.mean_squared_error(default_y_test, default_test_predicts)}\n\
    MAE = {metrics.mean_absolute_error(default_y_test, default_test_predicts)}\n\
    MAPE = {metrics.mean_absolute_percentage_error(default_y_test, default_test_predicts) * 100:.1f}%'
)

default_train_predicts = default_model.predict(default_x_train)
print(f'TRAIN:\n\
    MSE = {metrics.mean_squared_error(default_y_train, default_train_predicts)}\n\
    MAE = {metrics.mean_absolute_error(default_y_train, default_train_predicts)}\n\
    MAPE = {metrics.mean_absolute_percentage_error(default_y_train, default_train_predicts) * 100:.1f}%'
)


go.Figure(
    data=[
        go.Histogram(x=default_test_predicts, name='test'),
        go.Histogram(x=default_train_predicts, name='train'),
        go.Histogram(x=initial_df['source_attractiveness'], name='target'),
    ],
    layout=dict(title='Дефолтная модель')
).show()

TEST:
    MSE = 0.016299620949074823
    MAE = 0.0876673102316564
    MAPE = 154.7%
TRAIN:
    MSE = 0.01579951265222801
    MAE = 0.08671719704834088
    MAPE = 252.1%


Наша цель - сделать лучше

Для начала почистим данные:
- дропнем все `null`ы, потому что и без них датасет немаленький
- дропнем все отрицательные значения, где их не может быть
- добавим новую фичу - возраст
- приведем фичи к логичным типам *(клики, лайки, покупки и тд - к целым числам)*
- превратим '-' из количества жалоб в ноль *(допустим, что это типа пустое значение)*

In [21]:
df = initial_df.copy(deep=True)

print(f'До чистки данных: {df.shape}')

def setup_cleared_df(df):
    df_cleared = df.dropna().copy(deep=True)

    df_cleared.drop(df_cleared[df_cleared['clicks'] < 0].index, inplace=True)
    df_cleared.drop(df_cleared[df_cleared['likes'] < 0].index, inplace=True)
    df_cleared.drop(df_cleared[df_cleared['buys'] < 0].index, inplace=True)
    df_cleared.drop(df_cleared[df_cleared['4xx_errors'] < 0].index, inplace=True)
    df_cleared.drop(df_cleared[df_cleared['5xx_errors'] < 0].index, inplace=True)
    df_cleared.drop(df_cleared[df_cleared['average_dwelltime'] < 0].index, inplace=True)

    df_cleared['date_of_registration'] = pd.to_datetime(df_cleared['date_of_registration'])
    df_cleared['age'] = (pd.Timestamp('2024-09-30') - df_cleared['date_of_registration']) / pd.Timedelta(days=365)
    df_cleared.drop(['date_of_registration'], axis='columns', inplace=True)

    df_cleared['clicks'] = df_cleared['clicks'].astype('int32')
    df_cleared['likes'] = df_cleared['likes'].astype('int32')
    df_cleared['buys'] = df_cleared['buys'].astype('int32')

    df_cleared['complaints_count'] = df_cleared['complaints_count'].apply(non_numeric_to_minus_one)
    df_cleared.drop(df_cleared[df_cleared['complaints_count'] == -1].index, inplace=True)

    return df_cleared

df = setup_cleared_df(df)

print(f'После чистки данных: {df.shape}')

До чистки данных: (8000, 10)
После чистки данных: (4845, 10)


Посмотрим на некоторые распределения в датасете, чтобы понимать с чем имеем дело:

In [22]:
categories = df.groupby('category')
categories_count = categories.count().max(axis=1)
categories_likes = categories['likes'].sum()
categories_buys = categories['buys'].sum()
categories_clicks = categories['clicks'].sum()
categories_4xx = categories['4xx_errors'].sum()
categories_5xx = categories['5xx_errors'].sum()
categories_complaints = categories['complaints_count'].sum()

categories_attractiveness = categories['source_attractiveness']


categories_bars = make_subplots(
    rows=7,
    subplot_titles=[
        'Количество',
        'Кол-во кликов',
        'Кол-во лайков',
        'Кол-во покупок',
        'Кол-во ошибок',
        'Кол-во жалоб',
        'Привлекательность',
    ]
)

categories_bars.add_bar(
    name='Кол-во доменов',
    x=categories_count.index,
    y=categories_count.values,
    row=1, col=1,
)

categories_bars.add_bar(
    name='Кол-во кликов',
    x=categories_clicks.index,
    y=categories_clicks.values,
    row=2, col=1,
)

categories_bars.add_bar(
    name='Кол-во лайков',
    x=categories_likes.index,
    y=categories_likes.values,
    row=3, col=1,
)

categories_bars.add_bar(
    name='Кол-во покупок',
    x=categories_buys.index,
    y=categories_buys.values,
    row=4, col=1,
)
categories_bars.update_yaxes(type='log', range=[1, 10], row=4, col=1)

categories_bars.add_bar(
    name='Кол-во 4xx ошибок',
    x=categories_4xx.index,
    y=categories_4xx.values,
    row=5, col=1,
)
categories_bars.add_bar(
    name='Кол-во 5xx ошибок',
    x=categories_5xx.index,
    y=categories_5xx.values,
    row=5, col=1,
)
categories_bars.update_yaxes(type='log', row=5, col=1)

categories_bars.add_bar(
    name='Кол-во жалоб',
    x=categories_complaints.index,
    y=categories_complaints.values,
    row=6, col=1,
)

categories_bars.add_bar(
    name='Минимальная привлекательность',
    x=categories_attractiveness.min().index,
    y=categories_attractiveness.min().values,
    row=7, col=1,
)
categories_bars.add_bar(
    name='Средняя привлекательность',
    x=categories_attractiveness.mean().index,
    y=categories_attractiveness.mean().values,
    row=7, col=1,
)
categories_bars.add_bar(
    name='Максимальная привлекательность',
    x=categories_attractiveness.max().index,
    y=categories_attractiveness.max().values,
    row=7, col=1,
)

categories_bars.update_layout(
    title='Разбиение доменов по категориям',
    barmode='group',
    width=1000, height=2000,
)

categories_bars.show()

После просмотра распределений, появилось несколько идей:
- разбить домены по категории и для каждой категории натренировать отдельную модель *(пока лень и я хз как это неубого реализовать)*
- создать пару новых фич, связанных с пребыванием клиента на сайте:
  - активность - количество покупок + лайков за клик
  - опыт - насколько приятен был опыт взаимодействия с сайтом - отрицательное значение количества жалоб на сайт за клик
  - проведенное время - общее число *(минут?)* проведенных на сайте
- ~~забить на неважные фичи - ошибки, дата регистрации.~~ После пары попыток обучения, выяснилось, что "возраст" сайта и количество ошибок важны
- объединить ошибки в одну фичу *(просто для удобства, так как ошибки $\pm$ одинаковы по своей сути)*

Посмотрим что получается

In [23]:
def setup_train_df_activity(df):
    df_train_activity = df[[
        'age',
        'likes', 'buys',
        'category',
        'clicks',
        'complaints_count',
        'average_dwelltime',
    ]].copy(deep=True)

    df_train_activity['activity'] = (df_train_activity['likes'] + df_train_activity['buys']) / df_train_activity['clicks']
    df_train_activity['experience'] = - df_train_activity['complaints_count'] / df_train_activity['clicks']
    df_train_activity['time_spent'] = df_train_activity['average_dwelltime'] * df_train_activity['clicks']
    df_train_activity['errors'] = df['4xx_errors'] + df['5xx_errors']

    return df_train_activity

df_train_activity = setup_train_df_activity(df)

attractiveness = make_subplots(
    rows=2, cols=2,
    specs=[
        [{"colspan": 2}, None],
        [{}, {}],
    ]
)

attractiveness.add_scatter(
    x=df_train_activity['activity'], y=df['source_attractiveness'],
    mode='markers', marker=dict(size=1),
    name='Активность',
    row=1, col=1,
)
attractiveness.update_xaxes(row=1, col=1, title_text='Активность')
attractiveness.update_yaxes(row=1, col=1, title_text='Привлекательность')


attractiveness.add_scatter(
    x=df_train_activity['experience'], y=df['source_attractiveness'],
    mode='markers', marker=dict(size=1),
    name='Опыт',
    row=2, col=1,
)
attractiveness.update_xaxes(row=2, col=1, title_text='Опыт', range=[-0.5, 0.5])
attractiveness.update_yaxes(row=2, col=1, title_text='Привлекательность')


attractiveness.add_scatter(
    x=df_train_activity['time_spent'], y=df['source_attractiveness'],
    mode='markers', marker=dict(size=1.5),
    name='Время',
    row=2, col=2,
)
attractiveness.update_xaxes(row=2, col=2, title_text='Проведенное время')
attractiveness.update_yaxes(row=2, col=2, title_text='Привлекательность')


attractiveness.update(
    layout=dict(
        width=900, height=900,
    )
)
attractiveness.show()

Виднеется линейная зависимость, значит мы на верном пути

Натренируем модель

In [24]:
y = df['source_attractiveness']

# divide to test and train
x_train, x_test, y_train, y_test = train_test_split(
    df_train_activity, y,
    test_size=0.2,
    shuffle=True,
    stratify=df_train_activity['category']
)

# encode categories
encoder = OneHotEncoder(sparse_output=False)
x_train = np.hstack([x_train.drop(['category'], axis='columns'), encoder.fit_transform(x_train['category'].to_frame())])
x_test = np.hstack([x_test.drop(['category'], axis='columns'), encoder.transform(x_test['category'].to_frame())])

# normalizing
x_scaler = StandardScaler()
x_train = x_scaler.fit_transform(x_train)
x_test = x_scaler.transform(x_test)

Подберем гиперпараметры

In [25]:
import random
from random import randint


model = LinearRegression()
model.fit(x_train, y_train)

current_y_predict = model.predict(x_test)

mse = metrics.mean_squared_error(y_test, current_y_predict)

iterations = 100

for alpha in range(1, iterations):
    current_model = Ridge(alpha=alpha/iterations, random_state=randint(0, 4294967295))
    current_model.fit(x_train, y_train)

    current_y_predict = current_model.predict(x_test)
    current_mse = metrics.mean_squared_error(y_test, current_y_predict)

    if current_mse < mse:
        mse = current_mse
        model = current_model

for alpha in range(1, iterations):
    current_model = Lasso(alpha=alpha/iterations, random_state=randint(0, 4294967295))
    current_model.fit(x_train, y_train)

    current_y_predict = current_model.predict(x_test)
    current_mse = metrics.mean_squared_error(y_test, current_y_predict)

    if current_mse < mse:
        mse = current_mse
        model = current_model

for alpha in range(1, iterations):
    for ratio in range(1, iterations):
        current_model = ElasticNet(alpha=alpha/iterations, l1_ratio=ratio/iterations, random_state=randint(0, 4294967295))
        current_model.fit(x_train, y_train)

        current_y_predict = current_model.predict(x_test)
        current_mse = metrics.mean_squared_error(y_test, current_y_predict)

        if current_mse < mse:
            mse = current_mse
            model = current_model


test_predicts = model.predict(x_test)
print(f'TEST:\n\
    MSE = {metrics.mean_squared_error(y_test, test_predicts)}\n\
    MAE = {metrics.mean_absolute_error(y_test, test_predicts)}\n\
    MAPE = {metrics.mean_absolute_percentage_error(y_test, test_predicts) * 100:.1f}%\n\
    R2 = {metrics.r2_score(y_test, test_predicts)}'
)

train_predicts = model.predict(x_train)
print(f'TRAIN:\n\
    MSE = {metrics.mean_squared_error(y_train, train_predicts)}\n\
    MAE = {metrics.mean_absolute_error(y_train, train_predicts)}\n\
    MAPE = {metrics.mean_absolute_percentage_error(y_train, train_predicts) * 100:.1f}%\n\
    R2 = {metrics.r2_score(y_train, train_predicts)}'
)

go.Figure(
    data=[
        go.Histogram(x=test_predicts, name='test'),
        go.Histogram(x=train_predicts, name='train'),
        go.Histogram(x=initial_df['source_attractiveness'], name='target'),
    ],
    layout=dict(title='Конечная модель')
).show()

model

TEST:
    MSE = 0.00506926443065263
    MAE = 0.055932616800439906
    MAPE = 159.3%
    R2 = 0.8989909037000163
TRAIN:
    MSE = 0.004698362643232995
    MAE = 0.05260881957537276
    MAPE = 107.6%
    R2 = 0.9075009979307727


ElasticNet(alpha=0.01, l1_ratio=0.04, random_state=1189863010)

Для финальной модели немного изменим функции чистки данных, так как мы не можем просто взять и выкинуть какие-то входные данные

Вместо выкидывания - возьмем медианные значения новых фич, чтобы восстановить фичи с "плохими" значениями

In [85]:
def setup_not_cleared_df(df):
    df_not_cleared = df.copy(deep=True)

    df_not_cleared['date_of_registration'] = pd.to_datetime(df_not_cleared['date_of_registration'])
    df_not_cleared['age'] = (pd.Timestamp('2024-09-30') - df_not_cleared['date_of_registration']) / pd.Timedelta(days=365)
    df_not_cleared.drop(['date_of_registration'], axis='columns', inplace=True)

    df_not_cleared['clicks'] = df_not_cleared['clicks'].apply(non_numeric_to_zero).astype('int32')
    df_not_cleared['likes'] = df_not_cleared['likes'].apply(non_numeric_to_zero).astype('int32')
    df_not_cleared['complaints_count'] = df_not_cleared['complaints_count'].apply(lambda row: minus_to_zero_or_other(row, non_numeric_to_zero))
    df_not_cleared['average_dwelltime'] = df_not_cleared['average_dwelltime'].apply(non_numeric_to_zero)


    return df_not_cleared

def setup_df_activity(df):
    df_activity = df[[
        'age',
        'likes', 'buys',
        'category',
        'clicks',
        'complaints_count',
        'average_dwelltime',
    ]].copy(deep=True)

    df_activity.loc[df_activity['clicks'] == 0, 'clicks'] = ((df['likes'] + df['buys']) / df_train_activity['activity'].median()).astype('int32')
    df_activity.loc[df_activity['clicks'] == 0, 'clicks'] = 1

    df_activity['activity'] = (df_activity['likes'] + df_activity['buys']) / df_activity['clicks']
    df_activity['experience'] = - df_activity['complaints_count'] / df_activity['clicks']
    df_activity['time_spent'] = df_activity['average_dwelltime'] * df_activity['clicks']
    df_activity['errors'] = df['4xx_errors'] + df['5xx_errors']

    return df_activity

In [86]:
def predict_file_with_model(data_path, model):
    df = pd.read_csv(data_path)
    df = setup_not_cleared_df(df)
    df = setup_df_activity(df)

    x = np.hstack([df.drop(['category'], axis='columns'), encoder.transform(df['category'].to_frame())])
    x = x_scaler.transform(x)

    predict = model.predict(x)

    data = {'source_attractiveness': predict}
    submit = pd.DataFrame(data)
    submit.to_csv('submission.csv', index_label='ID')

def predict(data_path):
    predict_file_with_model(data_path, model)

predict(test_data_path)